## Energy mixes of the EU countries & UK

This notebooks prepares data for the ternary plot showing energy mixes of the EU countries and United Kingdom.

### Load prerequisities

(The prerequisities have to be already installed, e.g. install.packages("tidyverse")).

In [19]:
library(tidyverse)
library(ggtern)
library(svglite)

### Electricity generation data

Electricity data are loaded from Ember database. The csv file was downloaded into ``../data/ember/``. The original file may be accessed at [the following link](https://ember-climate.org/app/uploads/2022/07/yearly_full_release_long_format-1.csv).

[Ember](https://ember-climate.org/data-catalogue/yearly-electricity-data/) describes the dataset followingly: *The dataset contains yearly electricity generation, capacity, emissions, import and demand data for over 200 geographies. Data is collected from multi-country datasets (EIA, Eurostat, BP, UN) as well as national sources (e.g China data from the National Bureau of Statistics).*

As we will need only yearly electricity generation data for our anylysis, and we will examine only EU countries and UK, first we need to filter the dataset accordingly.

In [29]:
# Read and filter Ember data
electricity_eu_countries <-  read_csv("../data/ember/yearly_full_release_long_format-1.csv",
                # Select only columns relevant for the analysis.          
                        col_select = c("Country code", "Year", "Category", "Variable", "Value", "Unit", "EU"),
                        show_col_types = FALSE) |>
                # Filter EU countries and UK (= GBR), electricity generation category, and TWh unit.
                filter(EU == 1 | `Country code` == "GBR", Category %in% c("Electricity generation"),
                         Unit == "TWh") |>
                # Leave out the following columns: "Category", "Unit", and "EU", and select (and rename if needed) "Country code", "Year", "Variable" and "Value".
                select(CountryCode = "Country code",
                       Year = "Year",
                       Variable = "Variable",
                       ValueTWh = "Value") |>
                # Add a new column "ProdCategory" based on energy source.
                mutate(ProdCategory = case_when(Variable %in% c("Coal", "Gas", "Other Fossil") ~ "Fossil",
                                                Variable %in% c("Wind", "Solar", "Hydro", "Bioenergy") ~ "Renewables",
                                                Variable == "Nuclear" ~ "Nuclear")) |>
                # Filter out NAs in "ProdCategory" column, omit "Variable" column.
                filter(ProdCategory != "NA") |>
                select(-Variable) |>
                # Group rows by "CountryCode", "Year" and "ProdCategory", while summarising "ValueTWh" column.
                group_by(CountryCode, Year, ProdCategory) |>
                summarise(ValueTWh = sum(ValueTWh),
                          .groups = "drop")

In [30]:
head(electricity_eu_countries)

CountryCode,Year,ProdCategory,ValueTWh
<chr>,<dbl>,<chr>,<dbl>
AUT,2000,Fossil,16.41
AUT,2000,Nuclear,0.00
AUT,2000,Renewables,43.44
AUT,2001,Fossil,18.82
AUT,2001,Nuclear,0.00
AUT,2001,Renewables,42.24


As it will be convenient to include EU-27 total electricity generation per year and energy source category, we will add into the dataframe.

In [ ]:
# Calculate EU-27 total electricity generation (= "ValueTWh") per "Year" and "ProdCategory".
electricity_eu_total <- filter(electricity_eu_countries, CountryCode != "GBR") |>
  group_by(Year, ProdCategory) |>
  summarise(ValueTWh = sum(ValueTWh)) |>
  mutate(CountryCode = "EU27",
                   .before=Year)

# Append the total to the existing dataframe.
electricity_eu <- rbind(electricity_eu_countries, electricity_eu_total)

sadfdf

In [ ]:
# Calculate electricity shares.
electricity_shares <- electricity_eu |>
  group_by(CountryCode, Year) |>
  mutate(Total = sum(Value),
         Share = Value / Total) |>
  select(-Value) |>
  pivot_wider(names_from = ProdCategory, values_from = Share)

# Generate ternary plot as of 2021.

filter(electricity_shares, Year == 2021, CountryCode != "EU27") |>
  ggtern(aes(Nuclear, Fossil, Renewables)) +
  geom_point(aes(size = Total), colour = "#aecaff") +
  geom_text(aes(label = CountryCode), size = 3) +
  labs(x = "Nuclear",
       y = "Fossil",
       z = "Renewables",
       title = "Energy mixes in the EU countries",
       subtitle = "The larger the bubble, the higher the electricity generation") +
  theme_bw() +
  theme_nomask() +
  theme_showarrows()

# svglite::svglite("ternary-plot-eu.svg")
# dev.off()

# Generate ternary plot showing a path of a single country.

# filter(electricity_shares, CountryCode == "CZE") %>%
#   ggtern(aes(Nuclear, Fossil, Renewables)) +
#   geom_point(aes(colour = Year), size = 1) +
#   geom_path(aes(colour = Year)) +
#   labs(x = "Nuclear",
#        y = "Fossil",
#        z = "Renewables",
#        title = "Energy mix path in Czechia, 2000–2021") +
#   scale_colour_gradient("Rok") +
#   guides(colour = guide_colourbar(barheight = 8, barwidth = 0.5, title.position = "top")) +
#   theme_nomask() +
#   theme_bw() +
#   theme_showarrows()

# Generate small multiples of ternary plots for each country.
# electricity_shares %>%
#   ggtern(aes(Nuclear, Fossil, Renewables)) +
#   geom_point(aes(colour = Year), size = 1) +
#   geom_path(aes(colour = Year)) +
#   labs(x = "Jádro",
#        y = "Fosilní paliva",
#        z = "OZE",
#        title = "Vývoj podílu zdrojů na energetickém mixu v Dánsku") +
#   scale_colour_gradient("Rok") +
#   guides(colour = guide_colourbar(barheight = 8, barwidth = 0.5, title.position = "top")) +
#   facet_wrap(vars(CountryCode), ncol = 6) +
#   theme_nomask() +
#   theme_bw() +
#   theme_notitles() +
#   theme(text = element_blank())